In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
out_allinfo_HL = "AMC-TC-x001-HL-allinfo-mut.csv"
out_clones_HL = "AMC-TC-x001-HL-clones-mut.csv"

In [3]:
myfiles = [x for x in os.listdir() if x.endswith("-allinfo-filtered-mut.csv")]
myfiles

['AMC-TC-x001-AK1283_S0_L001.assembled-nomatch-IGH_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1283_S0_L001.assembled-nomatch-IGK_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1283_S0_L001.assembled-nomatch-IGL_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1284_S0_L001.assembled-nomatch-IGH_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1284_S0_L001.assembled-nomatch-IGK_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1284_S0_L001.assembled-nomatch-IGL_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1285_S0_L001.assembled-nomatch-IGH_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1285_S0_L001.assembled-nomatch-IGK_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1285_S0_L001.assembled-nomatch-IGL_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1286_S0_L001.assembled-nomatch-IGH_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1286_S0_L001.assembled-nomatch-IGK_HUMAN-allinfo-filtered-mut.csv',
 'AMC-TC-x001-AK1286_S0_L001.assembled-nomatch-IGL_HUMAN-allinfo-filtered-mu

In [4]:
def readFile(myfile):
    sample = myfile.split("_")[0]
    celltype = myfile.split("-")[-4]
    df = pd.read_csv(myfile)
    df["Sample"] = sample
    df["CellType"] = celltype
    df["chain"] = [x.split("_")[0] for x in df["CellType"]]
    df["CellID"] = [x.split("_")[0] for x in df["acc"]]
    return(df)

In [5]:
df = readFile(myfiles[0])
for myfile in myfiles[1:]:
    df_tmp = readFile(myfile)
    df = pd.concat([df, df_tmp])
df.tail()

,Unnamed: 0,acc,beforeMID,MID,afterMID,readingframe,cdr3pep,cdr3nuc,cdr3pepshort,cdr3nucshort,...,start.pos_y,end.pos_y,mut.count_y,mut.frac_y,align.length_y,align.seq_y,Sample,CellType,chain,CellID
979,979,TTTGCGCTCTTCATGT-1_contig_2,nomatch,nomatch,nomatch,2,CSSYTSSSTLLVFG,TGCAGCTCATATACAAGCAGCAGCACTCTCTTGGTGTTCGGC,CSSYTSS,TGCAGCTCATATACAAGCAGC,...,397.0,432.0,0.0,0.000000,35.0,===================================,AMC-TC-x001-AK1286,IGL_HUMAN,IGL,TTTGCGCTCTTCATGT-1
980,980,TTTGGTTGTGGTGTAG-1_contig_2,nomatch,nomatch,nomatch,0,CGTWDSSLSVWVFG,TGCGGAACATGGGATAGCAGCCTGAGTGTGTGGGTGTTCGGC,CGTWDSS,TGCGGAACATGGGATAGCAGC,...,387.0,424.0,0.0,0.000000,37.0,=====================================,AMC-TC-x001-AK1286,IGL_HUMAN,IGL,TTTGGTTGTGGTGTAG-1
981,981,TTTGTCAGTTTGCATG-1_contig_3,nomatch,nomatch,nomatch,0,CSSYTSSSLVFG,TGCAGCTCATATACAAGCAGCAGCCTGGTGTTCGGC,CSSYT,TGCAGCTCATATACA,...,395.0,430.0,0.0,0.000000,35.0,===================================,AMC-TC-x001-AK1286,IGL_HUMAN,IGL,TTTGTCAGTTTGCATG-1
982,982,TTTGTCATCCGCGGTA-1_contig_1,nomatch,nomatch,nomatch,1,CQSADSSNAYVFG,TGTCAATCAGCAGACAGTAGTAATGCTTATGTCTTCGGA,CQSADS,TGTCAATCAGCAGACAGT,...,384.0,422.0,1.0,0.026316,38.0,========================A=============,AMC-TC-x001-AK1286,IGL_HUMAN,IGL,TTTGTCATCCGCGGTA-1
983,983,TTTGTCATCGTGGACC-1_contig_1,nomatch,nomatch,nomatch,2,CAAWDDSLNAPVFG,TGTGCAGCATGGGATGACAGCCTGAATGCCCCGGTGTTCGGC,CAAWDDS,TGTGCAGCATGGGATGACAGC,...,400.0,435.0,0.0,0.000000,35.0,===================================,AMC-TC-x001-AK1286,IGL_HUMAN,IGL,TTTGTCATCGTGGACC-1


In [6]:
# List column names
df.columns

Index(['Unnamed: 0', 'acc', 'beforeMID', 'MID', 'afterMID', 'readingframe',
       'cdr3pep', 'cdr3nuc', 'cdr3pepshort', 'cdr3nucshort', 'cdr3_qual_min',
       'cdr3_qual_max', 'cdr3_qual_avg', 'cdr3_qual', 'nt_start', 'nt_end',
       'seq_length', 'V_flag', 'V_gene', 'J_flag', 'J_gene',
       'readingframe_seq', 'seq', 'pep', 'qual', 'V_sub', 'J_sub', 'V_main',
       'acc:1', 'nr_v_mains', 'nr_v_subs', 'nr_v_alleles', 'nr_j_subs',
       'nr_j_alleles', 'acc:2', 'readingframe:1', 'nr_sites', 'gene_x',
       'cigar_x', 'start.pos_x', 'end.pos_x', 'mut.count_x', 'mut.frac_x',
       'align.length_x', 'align.seq_x', 'gene_y', 'cigar_y', 'start.pos_y',
       'end.pos_y', 'mut.count_y', 'mut.frac_y', 'align.length_y',
       'align.seq_y', 'Sample', 'CellType', 'chain', 'CellID'],
      dtype='object')

In [7]:
# Get the cell barcode
f = lambda x: x.split("-")[0]
df["barcode"] = [x for x in map(f, df["CellID"])]

In [8]:
# Split data in two dataframes, one for heavy and one for the light chain
df_heavy = df[df["CellType"] == "IGH_HUMAN"]
df_light = df[df["CellType"] != "IGH_HUMAN"]
print("Heavy:", len(df_heavy))
print("Light:", len(df_light))

Heavy: 15765
Light: 15775


In [9]:
# Create short version of the CDR3's (consistent with IMGT)
df_heavy["cdr3pep_short"] = [x[:-7] for x in df_heavy["cdr3pep"]]
df_light["cdr3pep_short"] = [x[:-1] for x in df_light["cdr3pep"]]
#df_heavy.head()

/data/home/bioinfo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/data/home/bioinfo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [10]:
# add column with the annotation in the same format as cellranger does
def combine_chaininfo_cdr3 (chain, cdr3_short):
    return(chain.split("_")[0] + ":" + cdr3_short)
print(combine_chaininfo_cdr3("IGH_HUMAN", "CATHLIGSHFDYW"))

IGH:CATHLIGSHFDYW


In [11]:
df_heavy["chain_cdr3"] = [x for x in map(combine_chaininfo_cdr3, df_heavy["CellType"], df_heavy["cdr3pep_short"])]
#df_heavy.head()

/data/home/bioinfo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [12]:
df_light["chain_cdr3"] = [x for x in map(combine_chaininfo_cdr3, df_light["CellType"], df_light["cdr3pep_short"])]
#df_light.head()

/data/home/bioinfo/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
# Combine heavy and light chain info in one dataframe
df_HL_allinfo = pd.concat([df_heavy, df_light], sort=True)
df_HL_allinfo = df_HL_allinfo.sort_values(by="acc")
#df_HL_allinfo.head()

In [14]:
df_HL_allinfo.to_csv(out_allinfo_HL, index=None)
print("Wrote", out_allinfo_HL, "to disk")

Wrote AMC-TC-x001-HL-allinfo-mut.csv to disk


## Construct a clones file per cell

Put heavy and light chain info in one entry(row) per cell

In [15]:
cols = ["barcode", "chain_cdr3", "acc", "V_sub", "J_sub", "cdr3nuc", "cdr3pep", "mut.count_x", "mut.count_y", "mut.frac_x", "mut.frac_y"]
df_heavy = df_heavy[cols]
df_heavy = df_heavy.rename(columns={'chain_cdr3': 'cdr3_heavy', 'mut.count_x': 'mut.count.V.heavy', 'mut.count_y': 'mut.count.J.heavy', 'mut.frac_x': 'mut.frac.V.heavy', 'mut.frac_y': 'mut.frac.J.heavy'})
df_light = df_light[cols]
df_light = df_light.rename(columns={'chain_cdr3': 'cdr3_light', 'mut.count_x': 'mut.count.V.light', 'mut.count_y': 'mut.count.J.light', 'mut.frac_x': 'mut.frac.V.light', 'mut.frac_y': 'mut.frac.J.light'})
df_HL = pd.merge(df_heavy, df_light, how="inner", on="barcode")
df_HL.head()

,barcode,cdr3_heavy,acc_x,V_sub_x,J_sub_x,cdr3nuc_x,cdr3pep_x,mut.count.V.heavy,mut.count.J.heavy,mut.frac.V.heavy,...,cdr3_light,acc_y,V_sub_y,J_sub_y,cdr3nuc_y,cdr3pep_y,mut.count.V.light,mut.count.J.light,mut.frac.V.light,mut.frac.J.light
0,AAACCTGAGGCTAGGT,IGH:CAKDHSSWYEWYSLAPGDYW,AAACCTGAGGCTAGGT-1_contig_2,IGHV3-23,IGHJ4,TGTGCGAAAGATCACAGCAGCTGGTACGAATGGTATTCCCTCGCGC...,CAKDHSSWYEWYSLAPGDYWGQGTLVT,0.0,0.0,0.0,...,IGK:CQQYGSSPLFTF,AAACCTGAGGCTAGGT-1_contig_1,IGKV3-20,IGKJ3,TGTCAGCAGTATGGTAGCTCACCCTTATTCACTTTCGGC,CQQYGSSPLFTFG,0.0,0.0,0.000000,0.0
1,AAACCTGAGGCTAGGT,IGH:CAKDHSSWYEWYSLAPGDYW,AAACCTGAGGCTAGGT-1_contig_2,IGHV3-23,IGHJ4,TGTGCGAAAGATCACAGCAGCTGGTACGAATGGTATTCCCTCGCGC...,CAKDHSSWYEWYSLAPGDYWGQGTLVT,0.0,0.0,0.0,...,IGK:CQQYGSSPLFTF,AAACCTGAGGCTAGGT-1_contig_1,IGKV3-20,IGKJ3,TGTCAGCAGTATGGTAGCTCACCCTTATTCACTTTCGGC,CQQYGSSPLFTFG,0.0,0.0,0.000000,0.0
2,AAACCTGAGGCTAGGT,IGH:CAKDHSSWYEWYSLAPGDYW,AAACCTGAGGCTAGGT-1_contig_2,IGHV3-23,IGHJ4,TGTGCGAAAGATCACAGCAGCTGGTACGAATGGTATTCCCTCGCGC...,CAKDHSSWYEWYSLAPGDYWGQGTLVT,0.0,0.0,0.0,...,IGK:CQQYGSSPLFTF,AAACCTGAGGCTAGGT-1_contig_1,IGKV3-20,IGKJ3,TGTCAGCAGTATGGTAGCTCACCCTTATTCACTTTCGGC,CQQYGSSPLFTFG,0.0,0.0,0.000000,0.0
3,AAACCTGAGGCTAGGT,IGH:CAKDHSSWYEWYSLAPGDYW,AAACCTGAGGCTAGGT-1_contig_2,IGHV3-23,IGHJ4,TGTGCGAAAGATCACAGCAGCTGGTACGAATGGTATTCCCTCGCGC...,CAKDHSSWYEWYSLAPGDYWGQGTLVT,0.0,0.0,0.0,...,IGK:CQQYGSSPLFTF,AAACCTGAGGCTAGGT-1_contig_1,IGKV3-20,IGKJ3,TGTCAGCAGTATGGTAGCTCACCCTTATTCACTTTCGGC,CQQYGSSPLFTFG,0.0,0.0,0.000000,0.0
4,AAACCTGAGTGCGATG,IGH:CATHLIGSHFDYW,AAACCTGAGTGCGATG-1_contig_1,IGHV1-69,IGHJ4,TGTGCGACACACCTCATCGGGTCACACTTTGACTACTGGGGCCAGG...,CATHLIGSHFDYWGQGTLVT,0.0,0.0,0.0,...,IGK:CQQYDSYSTF,AAACCTGAGTGCGATG-1_contig_2,IGKV1-5,IGKJ2,TGCCAACAGTATGATAGTTATTCCACTTTTGGC,CQQYDSYSTFG,13.0,0.0,0.045775,0.0


In [16]:
df_HL.columns

Index(['barcode', 'cdr3_heavy', 'acc_x', 'V_sub_x', 'J_sub_x', 'cdr3nuc_x',
       'cdr3pep_x', 'mut.count.V.heavy', 'mut.count.J.heavy',
       'mut.frac.V.heavy', 'mut.frac.J.heavy', 'cdr3_light', 'acc_y',
       'V_sub_y', 'J_sub_y', 'cdr3nuc_y', 'cdr3pep_y', 'mut.count.V.light',
       'mut.count.J.light', 'mut.frac.V.light', 'mut.frac.J.light'],
      dtype='object')

In [17]:
# Group by cell barcode, keep all information in the case that there are multiple assigned light and/or heavy chains
f = lambda x: ";".join(list(set(x)))
df_HL_clones = df_HL.groupby("barcode").agg({
    "cdr3_heavy": ["nunique", f],
    "acc_x": ["nunique", f],
    "V_sub_x": ["nunique", f],
    "J_sub_x": ["nunique", f],
    "cdr3nuc_x": ["nunique", f],
    "cdr3pep_x": ["nunique", f],
    "cdr3_light": ["nunique", f],
    "acc_y": ["nunique", f],
    "V_sub_y": ["nunique", f],
    "J_sub_y": ["nunique", f],
    "cdr3nuc_y": ["nunique", f],
    "cdr3pep_y": ["nunique", f],
    "mut.count.V.heavy": [np.mean, np.median],
    "mut.count.J.heavy": [np.mean, np.median],
    "mut.count.V.light": [np.mean, np.median],
    "mut.count.J.light": [np.mean, np.median],
    "mut.frac.V.heavy": [np.mean, np.median],
    "mut.frac.J.heavy": [np.mean, np.median],
    "mut.frac.V.light": [np.mean, np.median],
    "mut.frac.J.light": [np.mean, np.median]})
df_HL_clones = df_HL_clones.reset_index()

In [18]:
df_HL_clones.head(50)

barcode cdr3_heavy  \
                        nunique   
0   AAACCTGAGGCTAGGT          1   
1   AAACCTGAGTGCGATG          1   
2   AAACCTGGTTCAGTAC          1   
3   AAACCTGGTTCATGGT          1   
4   AAACCTGTCCCGACTT          1   
5   AAACGGGAGCTCAACT          1   
6   AAACGGGAGGATGCGT          1   
7   AAACGGGAGTGTCTCA          1   
8   AAACGGGCACAGACAG          1   
9   AAACGGGCATTATCTC          1   
10  AAACGGGGTCTTCAAG          1   
11  AAACGGGTCGTGGACC          1   
12  AAAGATGAGACTCGGA          1   
13  AAAGATGAGTCCTCCT          1   
14  AAAGATGCAACAACCT          1   
15  AAAGATGCACAGCGTC          1   
16  AAAGATGCATCCAACA          1   
17  AAAGATGGTAGCTAAA          1   
18  AAAGATGGTCCAACTA          1   
19  AAAGATGGTGATAAAC          1   
20  AAAGATGGTGGTCTCG          1   
21  AAAGATGTCCTCGCAT          1   
22  AAAGCAAAGAGTTGGC          1   
23  AAAGCAAAGTTCGATC          1   
24  AAAGCAAGTGGACGAT          1   
25  AAAGCAAGTTATCCGA          2   
26  AAAGCAATCGATGAGG          2   
27  AAAGCAATCGGCGGTT          1   
28  AAAGCAATCGTACCGG          1   
29  AAAGCAATCTCACATT          2   
30  AAAGTAGAGAGGTAGA          1   
31  AAAGTAGAGCCAGTAG          1   
32  AAAGTAGAGCTGAAAT          1   
33  AAAGTAGAGTTGCAGG          1   
34  AAAGTAGCAGCGTTCG          1   
35  AAAGTAGCATCCTAGA          2   
36  AAAGTAGTCCCGGATG          1   
37  AAATGCCCAGACGCAA          1   
38  AAATGCCGTACAGCAG          1   
39  AAATGCCGTCTTCTCG          2   
40  AACACGTAGACGACGT          1   
41  AACACGTAGCATGGCA          1   
42  AACACGTAGCTCCCAG          1   
43  AACACGTCACCTATCC          1   
44  AACACGTCAGACAAAT          1   
45  AACACGTGTGTGACGA          2   
46  AACACGTGTTACCGAT          1   
47  AACACGTTCAGGTAAA          1   
48  AACACGTTCGATGAGG          1   
49  AACACGTTCTCCAGGG          2   

                                                        acc_x  \
                                             <lambda> nunique   
0                            IGH:CAKDHSSWYEWYSLAPGDYW       1   
1                                   IGH:CATHLIGSHFDYW       1   
2                              IGH:CARDQAIAAADLYYFDYW       2   
3                                 IGH:CARLGVGATGVYDYW       2   
4                             IGH:CARDLSWGVVANPLTGHYW       2   
5                                 IGH:CAREYSSYEGDFDLW       1   
6                                    IGH:CARGQWSYSDYW       2   
7                          IGH:CATVRITMVRGVIIYDRTFDYW       2   
8                               IGH:CARDRTQQLASNWFDPW       2   
9                              IGH:CARATAPYYYDSSGYLPW       1   
10                      IGH:CARDQGEELRFLEWLPYYYYGMDVW       2   
11                       IGH:CARDVGGLLWFGESSSYYYYMDVW       1   
12                        IGH:CATVSPSYCSSTSCYTISWFDPW       1   
13           IGH:CARDSNPTLPSTRPTYYDFWSGYLGEYYYYYGMDVW       1   
14                                IGH:CAKDPSGPFNGWEYW       2   
15                            IGH:CAKVIGGAVLWFGETFDIW       2   
16                           IGH:CARDYYDSSGYYLHWYFDLW       2   
17                              IGH:CARDPRWGYTSSGFDSW       1   
18                                  IGH:CARPSQNWGLDYW       2   
19                       IGH:CARASVGYDILTGYYYYYYGMDVW       1   
20                      IGH:CARDGGVCSSTSCYWDYYYYGMDVW       2   
21                              IGH:CAKDVYGRRLRGWFDPW       1   
22                 IGH:CARHYSRPYETLIMITFGGVIAREREADYW       1   
23                                 IGH:CARMRGGVPWFDPW       2   
24                              IGH:CARDWDSSSWTNWFDPW       2   
25       IGH:CARAGGNWNYHGGAHNYFDPW;IGH:CARMGVGTGAFDIW       3   
26   IGH:CAKSPATGQTYYYYYYMDVW;IGH:CAKSPATGQTYYYYYMDVW       2   
27                             IGH:CARKLGDQLWLKDGMDVW       1   
28                           IGH:CAREHGGPWPAVYYYGMDVW       2   
29             IGH:CARRVLNGFGFDYW;IGH:CAAAVGMQYYGMDVW       4   
30                                  IGH:CAKGKWDDAFEIW       1   
31                                  IGH:CARGPGG

In [19]:
df_HL_clones = df_HL_clones.rename(columns={'<lambda>': 'concat'})
df_HL_clones.columns = ["_".join(x) for x in df_HL_clones.columns] # flatten column levels
df_HL_clones = df_HL_clones.rename(columns={'barcode_': 'barcode'})
df_HL_clones.head()

,barcode,cdr3_heavy_nunique,cdr3_heavy_concat,acc_x_nunique,acc_x_concat,V_sub_x_nunique,V_sub_x_concat,J_sub_x_nunique,J_sub_x_concat,cdr3nuc_x_nunique,...,mut.count.J.light_mean,mut.count.J.light_median,mut.frac.V.heavy_mean,mut.frac.V.heavy_median,mut.frac.J.heavy_mean,mut.frac.J.heavy_median,mut.frac.V.light_mean,mut.frac.V.light_median,mut.frac.J.light_mean,mut.frac.J.light_median
0,AAACCTGAGGCTAGGT,1,IGH:CAKDHSSWYEWYSLAPGDYW,1,AAACCTGAGGCTAGGT-1_contig_2,1,IGHV3-23,1,IGHJ4,1,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
1,AAACCTGAGTGCGATG,1,IGH:CATHLIGSHFDYW,1,AAACCTGAGTGCGATG-1_contig_1,1,IGHV1-69,1,IGHJ4,1,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.021391,0.006734,0.0,0.0
2,AAACCTGGTTCAGTAC,1,IGH:CARDQAIAAADLYYFDYW,2,AAACCTGGTTCAGTAC-1_contig_1;AAACCTGGTTCAGTAC-1...,1,IGHV3-30-3,1,IGHJ4,1,...,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
3,AAACCTGGTTCATGGT,1,IGH:CARLGVGATGVYDYW,2,AAACCTGGTTCATGGT-1_contig_2;AAACCTGGTTCATGGT-1...,1,IGHV4-59,1,IGHJ4,1,...,0.0,0.0,0.001145,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
4,AAACCTGTCCCGACTT,1,IGH:CARDLSWGVVANPLTGHYW,2,AAACCTGTCCCGACTT-1_contig_1;AAACCTGTCCCGACTT-1...,1,IGHV3-48,1,IGHJ4,1,...,0.0,0.0,0.003378,0.003378,0.0,0.0,0.000000,0.000000,0.0,0.0


In [20]:
print("Cells all:", len(df_HL_clones))
print("Cells with 1 heavy and 1 light chain:", len(df_HL_clones[(df_HL_clones['cdr3_heavy_nunique'] == 1) & (df_HL_clones['cdr3_light_nunique'] == 1)]))

Cells all: 2852
Cells with 1 heavy and 1 light chain: 2368


In [21]:
df_HL_clones.to_csv(out_clones_HL, index=None)
print("Wrote", out_clones_HL, "to disk")

Wrote AMC-TC-x001-HL-clones-mut.csv to disk
